# DOCUMENTAÇÃO

##Descrição

- Trabalho: Techchallenge (Módulo 1)
- Grupo: 8
- Participantes:

   Wesley Estevão dos santos - RM352704 - wesley.estevao@outlook.com
  
   Márcio Garrido Estácio - RM352740 -  marciogarrido2007@yahoo.com.br
  
   Luis Philipe Micioneiro Bezerra - RM352472 - micioneiro.bezerra@gmail.com
  
   Andre Taets Jorge - RM353109 - andre.taets@gmail.com

   Amanda Farias - RM352620 - amanda_ramos16@hotmail.com
-Objetivo:

   Analisar e criar empresa, trazendo resumo e storytelling dos dados de exportação de vinhos com base no [Banco de dados](http://vitibrasil.cnpuv.embrapa.br/index.php?opcao=opt_01) da embrapa disponibilizado pela FIAP.


## Código (ETL)

Bibliotecas utilizadas:
 ```
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
```
Função para que os dados do DataSet sejam formatados:

Será ajustado sem notação cientifica.

pd.options (nas opcoes) display (mostra) float_float (flutuante)

`pd.options.display.float_format = "{:.2f}".format`

[Base de dados da embrapa](http://vitibrasil.cnpuv.embrapa.br/index.php?opcao=opt_01)

`df = pd.read_csv('https://raw.githubusercontent.com/wesleyesantos/Techchanlleng/main/ExpVinho.csv', sep=';')`


Analisando tamanho da tabela

`df.shape`

Anasando tipos de dados da tabela

`df.info()`

Selecionar últimos 15 anos

Análise das colunas existentes para separar valores de litros

`df.columns`

Separando colunas índices das colunas que serão alteradas

```
coluna_1 = df.columns[:2]
coluna_2 = df.columns[78:]
```
Separando os últimos 15 anos para a análise

```
colunas = coluna_1.append(coluna_2)
colunas[2:]
```

Ajustando o dataframe principal somente com colunas que serão utilizadas

`df = df[colunas]`

ID não é necessário, vamos excluir:

`df = df.drop('Id',axis=1)`

Adicionando país como índice principal do dataframe

`df = df.set_index('País')`

Verificando tipos das colunas, após ajuste

`dados.info()`

Retirando índice da coluna país

`df = df.reset_index()`

Criando dois df dados_qtd | dados_valores

```
colunas_df = df.columns
colunas_df[:5]
```

Criação de função para para calcular as quantidade dos litros e os valores


```
valores_usd = []
quantidade_l = []

for i in range(0,len(colunas_df)):
    if colunas_df[i] == 'País':
        valores_usd.append(colunas_df[i])
        quantidade_l.append(colunas_df[i])
    elif '.1' in colunas_df[i]:
        valores_usd.append(colunas_df[i])
    else:
        quantidade_l.append(colunas_df[i])
```

Separação de quantidades e valores

```
df_valores = df[valores_usd]
df_qtd = df[quantidade_l]
```

Ajuste dos nomes das colunas

`df_valores.columns = df_valores.columns.str.replace('\.1','', regex=True)`

Utilizado o Melt para realizar a transposição dos valores

```
df_valores_norm = df_valores.reset_index().melt(id_vars=['País'], value_vars=df_valores.columns)
df_valores_norm.columns = ['pais_destino','ano','valor_usd']
df_valores_norm
```

Utilizado o Melt para realizar a transposição dos valores

```
df_qtd_norm = df_qtd.reset_index().melt(id_vars=['País'], value_vars=df_qtd.columns)
df_qtd_norm.columns = ['pais_destino','ano','litros']
df_qtd_norm
```

Ordenação de valores da quantidade de litros

`df_qtd_norm.sort_values(by='litros', ascending=False).head(10)`

Juntando a coluna de valor na nossa tabela matriz

`df = df_qtd_norm.join(df_valores_norm['valor_usd'])`

Criação de dataframe para pegar os nomes dos países e realizar ajustes

`lista_paises = df['pais_destino'].unique()`

Criação de dicionário para realizar devidas correções nos países

```
correcoes = {
    'Alemanha, República Democrática': 'Alemanha',
    'Antilhas Holandesas':'Antilhas Holandesas,Países Baixos',
    'Aruba': 'Aruba,Países Baixos',
    'Bósnia-Herzegovina': 'Bósnia-Herzegovina,Bosnia e Herzegovina',
    'Cayman, Ilhas': 'Ilhas Cayman',
    'Chipre': 'Chipre',
    'Cocos (Keeling), Ilhas': 'Cocos (Keeling), Ilhas ,Austrália',
    'Congo':'Congo,Países Baixos',
    'Coreia, Republica Sul': 'Coreia do Sul',
    'Curaçao':'Curaçao,Países Baixos',
    'Emirados Arabes Unidos': 'Emirados Árabes Unidos',
    'Eslovaca, Republica': 'Eslováquia',
    'Gibraltar':'Gibraltar,Reino Unido',
    'Guiana Francesa': 'Guiana Francesa',
    'Guine Bissau': 'Guiné-Bissau',
    'Guine Equatorial': 'Guiné Equatorial',
    'Hong Kong': 'Hong Kong',
    'Ilha de Man': 'Ilha de Man',
    'Indonésia': 'Indonésia',
    'Irã': 'Irã',
    'Países Baixos': 'Países Baixos',
    'Singapura': 'Singapura',
    'Taiwan (FORMOSA)': 'Taiwan',
    'Tcheca, República': 'República Tcheca',
    'Trinidade Tobago': 'Trinidade e Tobago',
    'Guiana Francesa':'Guiana Francesa,França',
    'Hong Kong':'Hong Kong,China',
    'Taiwan (FORMOSA)':'Taiwan,China'
}
```
Correção de países

`lista_paises_corrigida = [correcoes.get(pais, pais) for pais in lista_paises]`

Fazendo o replace dos nomes de países:

`df['pais_destino'] = df['pais_destino'].replace(correcoes)`

Inclusão da Coluna pais_origem = 'Brasil'

`df['pais_origem'] = 'Brasil'`

Organização da ordem das colunas

```
ordem_colunas = ['pais_origem','pais_destino','ano','litros','valor_usd']
df = df[ordem_colunas]
```

Remoção das linhas com valores zerados

```
df = df[(df != 0).all(axis =1)]
```

Inclusão de dia e mês para criação da data

```
df['dia'] = 1
df['mes'] = 1
```

inclusão da coluna data

`df['data'] = pd.to_datetime(df[['ano', 'mes', 'dia']].astype(str).agg('-'.join, axis=1), format='%Y-%m-%d')`

retirada das demais colunas que não serão mais necessárias agora que temos a data (ano, mes e dia)

` df = df.drop(['ano', 'mes', 'dia'], axis =1)`

### DADOS EXTERNOS - Economia

Variação do dólar

Banco de dados

`dolar = pd.read_csv('https://github.com/wesleyesantos/Techchalleng/raw/main/dolar.csv')`

Análise dos tipos e valores nulos

`dolar.info()`

Limpeza da coluna vol que está vazia

`dolar.drop('vol', axis = 1)`

Ajuste do campo data

`dolar['data'] = pd.to_datetime(dolar['data'])`


Ajuste do tipo da coluna variação que devido estar com sinal de % seu tipo era string, e agora a ajustamos para valor flutuante (float) e ordenamos pelo valor de variação

```
dolar.replace('%','',inplace=True,regex=True)
dolar['var'] = (dolar['var']).astype('float64')
dolar = dolar.sort_values(by='var',ascending=False)
```

Inclusão das colunas ano e mês e ordenação dos dados por esses dois campos

```
dolar['ano']= dolar['data'].dt.year
dolar['meses'] = dolar['data'].dt.month
dolar = dolar.sort_values(by=['ano','meses'])
```

ajuste do tipo da coluna ultimo para valor flutuante

`dolar['ultimo'] = dolar['ultimo'].astype('float64')`

### DADOS EXTERNOS - Inclusão

(Mulheres no mundo dos vinhos)

Pesquisa feita pela Universidade Federal Rural do Rio de janeiro, feita com 200 mulheres demonstrou os seguintes dados da tabela inclusão.

`inclusao = pd.DataFrame({'classificacao': ['Empresárias', 'Sommelières', 'Estudantes', 'Outros'],'quantidade': [77, 49, 21, 53]})`

### DADOS EXTERNOS - Demografia
(População Maior Idade) - Com base no Top 10 Exportadores

Inclusão de População histórica por país:

Fonte: https://paises.ibge.gov.br/#/mapa
Adicionando percentual de faixa etária dos países listados.

Faixa de Idade que permite consumo de vinho: https://vaocubo.com/2020/05/30/idade-minima-legal-para-comprar-ou-beber-vinho-ao-redor-do-mundo/ Base utilizada de piramide etária: https://www.populationpyramid.net/

    - Sem Idade: China - 100%
    - +16/17 anos: Alemanha - 84% da população | Haiti - 65,8% da população | Espanha - 85% da população
    - +18/19 anos: Russia - 78,2% | Holanda (Países Baixos) - 80,8% | Inglaterra (Reino Unido) - 78,7%
    - +20 anos: Paraguai - 62,3% | Japão - 84%
    - +21 anos: Estados Unidos - 74%

```
populacao = pd.DataFrame([['Rússia', 2022, 143555736, 78.2],
                       ['Rússia', 2021, 143449286, 78.2],
                       ['Rússia', 2020, 144073139, 78.2],
                       ['Rússia', 2019, 144406261, 78.2],
                       ['Rússia', 2018, 144477859, 78.2],
                       ['Rússia', 2017, 144496739, 78.2],
                       ['Rússia', 2016, 144342397, 78.2],
                       ['Rússia', 2015, 144096870, 78.2],
                       ['Rússia', 2014, 143819667, 78.2],
                       ['Rússia', 2013, 143506995, 78.2],
                       ['Rússia', 2012, 143201721, 78.2],
                       ['Rússia', 2011, 142960908, 78.2],
                       ['Rússia', 2010, 142849468, 78.2],
                       ['Rússia', 2009, 142785349, 78.2],
                       ['Rússia', 2008, 142742366, 78.2],

                       ['Paraguai', 2022, 6780744, 62.3],
                       ['Paraguai', 2021, 6703799, 62.3],
                       ['Paraguai', 2020, 6618695, 62.3],
                       ['Paraguai', 2019, 6530026, 62.3],
                       ['Paraguai', 2018, 6443328, 62.3],
                       ['Paraguai', 2017, 6355404, 62.3],
                       ['Paraguai', 2016, 6266615, 62.3],
                       ['Paraguai', 2015, 6177950, 62.3],
                       ['Paraguai', 2014, 6090721, 62.3],
                       ['Paraguai', 2013, 6005652, 62.3],
                       ['Paraguai', 2012, 5923322, 62.3],
                       ['Paraguai', 2011, 5843939, 62.3],
                       ['Paraguai', 2010, 5768613, 62.3],
                       ['Paraguai', 2009, 5702574, 62.3],
                       ['Paraguai', 2008, 5645148, 62.3],

                       ['Estados Unidos', 2022, 333287557, 74],
                       ['Estados Unidos', 2021, 332031554, 74],
                       ['Estados Unidos', 2020, 331511512, 74],
                       ['Estados Unidos', 2019, 328329953, 74],
                       ['Estados Unidos', 2018, 326838199, 74],
                       ['Estados Unidos', 2017, 325122128, 74],
                       ['Estados Unidos', 2016, 323071755, 74],
                       ['Estados Unidos', 2015, 320738994, 74],
                       ['Estados Unidos', 2014, 318386329, 74],
                       ['Estados Unidos', 2013, 316059947, 74],
                       ['Estados Unidos', 2012, 313877662, 74],
                       ['Estados Unidos', 2011, 311583481, 74],
                       ['Estados Unidos', 2010, 309327143, 74],
                       ['Estados Unidos', 2009, 306771529, 74],
                       ['Estados Unidos', 2008, 304093966, 74],

                       ['China', 2022, 1412175000, 100],
                       ['China', 2021, 1412360000, 100],
                       ['China', 2020, 1411100000, 100],
                       ['China', 2019, 1407745000, 100],
                       ['China', 2018, 1402760000, 100],
                       ['China', 2017, 1396215000, 100],
                       ['China', 2016, 1387790000, 100],
                       ['China', 2015, 1379860000, 100],
                       ['China', 2014, 1371860000, 100],
                       ['China', 2013, 1363240000, 100],
                       ['China', 2012, 1354190000, 100],
                       ['China', 2011, 1345035000, 100],
                       ['China', 2010, 1337705000, 100],
                       ['China', 2009, 1331260000, 100],
                       ['China', 2008, 1324655000, 100],

                       ['Espanha', 2022, 47615034, 85],
                       ['Espanha', 2021, 47415750, 85],
                       ['Espanha', 2020, 47365655, 85],
                       ['Espanha', 2019, 47134837, 85],
                       ['Espanha', 2018, 46797754, 85],
                       ['Espanha', 2017, 46593236, 85],
                       ['Espanha', 2016, 46484062, 85],
                       ['Espanha', 2015, 46444832, 85],
                       ['Espanha', 2014, 46480882, 85],
                       ['Espanha', 2013, 46620045, 85],
                       ['Espanha', 2012, 46773055, 85],
                       ['Espanha', 2011, 46742697, 85],
                       ['Espanha', 2010, 46576897, 85],
                       ['Espanha', 2009, 46362946, 85],
                       ['Espanha', 2008, 45954106, 85],

                       ['Haiti', 2022, 11584996, 65.8],
                       ['Haiti', 2021, 11447569, 65.8],
                       ['Haiti', 2020, 11306801, 65.8],
                       ['Haiti', 2019, 11160438, 65.8],
                       ['Haiti', 2018, 11012421, 65.8],
                       ['Haiti', 2017, 10863543, 65.8],
                       ['Haiti', 2016, 10713849, 65.8],
                       ['Haiti', 2015, 10563757, 65.8],
                       ['Haiti', 2014, 10412740, 65.8],
                       ['Haiti', 2013, 10261206, 65.8],
                       ['Haiti', 2012, 10108539, 65.8],
                       ['Haiti', 2011, 9954312, 65.8],
                       ['Haiti', 2010, 9842880, 65.8],
                       ['Haiti', 2009, 9730638, 65.8],
                       ['Haiti', 2008, 9575247, 65.8],


                       ['Reino Unido', 2022, 66971411, 78.7],
                       ['Reino Unido', 2021, 67026300, 78.7],
                       ['Reino Unido', 2020, 67081234, 78.7],
                       ['Reino Unido', 2019, 66836327, 78.7],
                       ['Reino Unido', 2018, 66460344, 78.7],
                       ['Reino Unido', 2017, 66058859, 78.7],
                       ['Reino Unido', 2016, 65611593, 78.7],
                       ['Reino Unido', 2015, 65116219, 78.7],
                       ['Reino Unido', 2014, 64602298, 78.7],
                       ['Reino Unido', 2013, 64128273, 78.7],
                       ['Reino Unido', 2012, 63700215, 78.7],
                       ['Reino Unido', 2011, 63258810, 78.7],
                       ['Reino Unido', 2010, 62766365, 78.7],
                       ['Reino Unido', 2009, 62276270, 78.7],
                       ['Reino Unido', 2008, 61806995, 78.7],

                       ['Países Baixos', 2022, 17703090, 80.8],
                       ['Países Baixos', 2021, 17533044, 80.8],
                       ['Países Baixos', 2020, 17441500, 80.8],
                       ['Países Baixos', 2019, 17344874, 80.8],
                       ['Países Baixos', 2018, 17231624, 80.8],
                       ['Países Baixos', 2017, 17131296, 80.8],
                       ['Países Baixos', 2016, 17030314, 80.8],
                       ['Países Baixos', 2015, 16939923, 80.8],
                       ['Países Baixos', 2014, 16865008, 80.8],
                       ['Países Baixos', 2013, 16804432, 80.8],
                       ['Países Baixos', 2012, 16754962, 80.8],
                       ['Países Baixos', 2011, 16693074, 80.8],
                       ['Países Baixos', 2010, 16615394, 80.8],
                       ['Países Baixos', 2009, 16530388, 80.8],
                       ['Países Baixos', 2008, 16445593, 80.8],

                       ['Alemanha', 2022, 84079811, 84],
                       ['Alemanha', 2021, 83196078, 84],
                       ['Alemanha', 2020, 83160871, 84],
                       ['Alemanha', 2019, 83092962, 84],
                       ['Alemanha', 2018, 82905782, 84],
                       ['Alemanha', 2017, 82657002, 84],
                       ['Alemanha', 2016, 82348669, 84],
                       ['Alemanha', 2015, 81686611, 84],
                       ['Alemanha', 2014, 80982500, 84],
                       ['Alemanha', 2013, 80645605, 84],
                       ['Alemanha', 2012, 80425823, 84],
                       ['Alemanha', 2011, 80274983, 84],
                       ['Alemanha', 2010, 81776930, 84],
                       ['Alemanha', 2009, 81902307, 84],
                       ['Alemanha', 2008, 82110097, 84],

                       ['Japão', 2022, 125124989, 84],
                       ['Japão', 2021, 125681593, 84],
                       ['Japão', 2020, 126261000, 84],
                       ['Japão', 2019, 126633000, 84],
                       ['Japão', 2018, 126811000, 84],
                       ['Japão', 2017, 126972000, 84],
                       ['Japão', 2016, 127076000, 84],
                       ['Japão', 2015, 127141000, 84],
                       ['Japão', 2014, 127276000, 84],
                       ['Japão', 2013, 127445000, 84],
                       ['Japão', 2012, 127629000, 84],
                       ['Japão', 2011, 127833000, 84],
                       ['Japão', 2010, 128070000, 84],
                       ['Japão', 2009, 128047000, 84],
                       ['Japão', 2008, 128063000, 84]],
                       columns = ['pais_destino', 'ano', 'populacao', 'percentual_maior_idade']
                     )
```

incluindo colunas dia e mês para inclusão da coluna data

```
populacao['dia'] = 1
populacao['mes'] = 1
populacao['data'] = pd.to_datetime(populacao[['ano', 'mes', 'dia']].astype(str).agg('-'.join, axis=1), format='%Y-%m-%d')
```

Exclusão das colunas dia, mês e ano, devido já ter os dados na coluna data

`populacao = populacao.drop(['ano', 'mes', 'dia'],axis = 1 )`

## Power BI (DATAVIZ e STORYTELLING)


### Fonte de dados - Sript Python

Utilizamos o código PY subindo direto como script do Power BI, com isso ele nos trouxe todos os dataframes gerados, fizemos o possível para no ETL realizar a criação da menor quantidade de dataframes e facilitar o filtro da base de dados no Dashboard.

### Estrutura dos dashboard

---

Card_HTML

Informação  do campo:
Criamos essa visualização para mostrar a soma e a média do valor USD, onde foi utilizado a visualização HTML dentro do power BI gerando o código abaixo.


Medida:
Incluimos 4 variáveis onde 2 são títulos e outras duas são os calculos realizados nas medidaas [Valor_total] (Que traz a soma do valor USD) e [Valor_meio] (Que traz a média dos valores exportados em dólar) e com isso solicitamos que a variação retornasse o código HTML que define inicialmente os estilos cores e fontes e em seguida define o corpo da visualização.

```
		Card_HTML = -- Código HTML para gerar o CARD
Var vTitulo_1 = "Valor Exportado em US$" -- Título do primeiro cálculo
Var vTitulo_2 = "Valor Médio:" -- Título do segundo cálculo
Var vMedida_1 = FORMAT([Valor_total], "#,##0") -- Medida que calcula o valor (USD) total
Var vMedida_2 = FORMAT([Valor_medio], "#,##0") -- Medida que calcula média do valor (USD)

Return


-- ÍNICIO FORMATAÇÕES
"<style>
:root
    {
        --family_fonte: calibri;
        --background_color_1: #D7D2CC;
        --background_color_2: #D7D2CC;
        --background_border: 8;    
        --wave_color: #D7D2CC;
        --wave_opacity: 0.06; /* valores de 0 à 1*/
        --cor_line: #f1f1f1;
        --title_1_color: #141E28;
        --title_1_font_size: 17;
        --measure_1_color: #B06F77;
        --measure_1_font_size: 40;
        --title_2_color: #141E28;
        --title_2_font_size: 16;
        --measure_2_color: #B06F77;
        --measure_2_font_size: 16;
        --measure_2_posicionamento: 0%;
    }
     /* *****************************************************
    FINAL FORMATAÇÕES
    ***************************************************** */
</style>
_
<body>    
   <svg viewBox='0 0 300 150' fill='none'>
   _
   <rect id='bg' width='300' height='130' rx='8' fill='url(#colorGrad'/>
   _
   <defs> <linearGradient id='colorGrad' x1='0' y1='0' x2='1' y2='1' > <stop offset='0' stop-color='var(--background_color_1)'/> <stop offset='1' stop-color='var(--background_color_2)'/> </linearGradient> </defs>
   _
   <text id='title_1' fill='var(--title_1_color)' font-family='var(--family_fonte)' font-size='var(--title_1_font_size)'> <tspan x='25' y='37'>" & vTitulo_1 & "</tspan> </text>
   _
   <text id='measure_1' fill='var(--measure_1_color)' font-family='var(--family_fonte)' font-size='var(--measure_1_font_size)' font-weight='bold'> <tspan x='27' y='72'>" & vMedida_1 & "</tspan></text>
   _
   <text id='title_2' fill='var(--title_2_color)' font-family='var(--family_fonte)' font-size='var(--title_2_font_size)'> <tspan x='142' y='111'>" & vTitulo_2 & "</tspan> </text>
   _
   <text id='measure_2' f fill='var(--measure_2_color)' font-family='var(--family_fonte)' font-size='var(--measure_2_font_size)' font-weight='bold'> <tspan x='230' y='111'>"&vMedida_2&"</tspan> </text>
   _
   <line id='linha' x1='113' y1='91' x2='300' y2='91' stroke='var(--cor_line)' stroke-opacity='0.5' stroke-width='0.5'/>
   </svg>
</body>
"
```

--------------


Gráfico "Litros Exportados X Continente"

Informação do Campo:

Mostrar o cenário global quais são os continentes que mais adquirem vinhos por exportações da Celestial.

Estrutura do gráfico:

	eixo x = foi realizado uma medida dinâmica de soma a partir do campo litros
	eixo y = coluna continentes (Que foi adquirida via dados externos e incluso em nosso ETL.)

----------

Gráfico "Mapa":

Informação do Campo:

Criado um gráfico para demonstrar o quanto o nosso vinho é reconhecido mundialmente, de modo a mostrasr que conseguimos vender para todo o globo.
Neste mapa, temos o uso da medida "Litros_total" x pais_destino, a função de representar a quantidade de litros exportados está no "tamanho da bolha", quanto maior, mais exportações o país realizou.

Estrutura do gráfico:

	Localização = Utilizamos o país destino pra distribuir a informação pode localidade

	Tamanho da bolha = Para o tamanho da bolha realizamos a medida Litro_total onde foi realizado uma soma da coluna litros

	Litro_total = SUMX(df, df[litros])

---------------

Grafico "TOP 10 Paises X Litros Exportados"

Informação do campo:

O gráfico mostra os países que mais adquiriram litros de vinho no período (últimos 15 anos), foi utilizado como informação a "soma dos litros" tratados no df (Dataframe).

Estrutura do gráfico:

	Y = pais_destino
	X = media Litro total

e foi realizado um filtro top N pra selecionar os 10 primeiros na informação de pais_destino e considerando o agrupamento da soma.

---------------

Tabela "TOP 20 - País de Destino Exportações"

Informação do campo:

A tabela mostra a coluna pais_destino e seus totais de exportação em litros e qual foi a receita gerada em USD no período analisado.

Estrutura da tabela:
Soma total do faturamento em US$:

	SUMX(df, df[valor_usd])

Soma do total de litros exportados:

	Sumx( df, df[litros])

e foi aplicado um filtro dos top 20 por total faturado nas exportações.

-------------

Gráfico "Faturamento ano a ano"

Informação do Campo:

Gráfico para obtenção de insights e entendimento do panorama geral da exportação de vinhos e lucratividade no período. Com base neste gráfico, podemos verificar períodos de alto retorno financeiro e baixas nas vendas, além disso, conseguimos também verificar que nos últimos anos, estamos mantendo uma certa consistências nas vendas e períodos de aumento no lucro.

Estrutura do gráfico:
	Eixo x =  Ano
	Eixo y: [Valor_total] = SUMX(df, df[valor_usd])

---------------------------------

Gráfico "Paraguai" / "Rússia"

Informação Paraguai:
Aprofundamento na análise deste país (Paraguai), em virtude da alta representatividade que o mesmo apresentou ao longo dos anos, a importação segue consistente e nos ultimos 10 anos, apresenta leve aumento no comparativo ano a ano.

Informação Rússia:

Aprofundamento na análise deste país (Rússia), temos valores elevados de exportação em anos especificos seguidos de períodos sem exportação, trata-se de um país que não apresenta consistencia de exportação, há muitos ruídos politicos envolvidos em negociações com esse país, fato este que pode explicar períodos sem exportação.

Estrutura do gráfico:

	X = data(year)
	Y = Litro_total = SUMX(df, df[litros])
	Múltiplos gráficos = pais_destino (Paraguai e Rússia)

Realizado filtro topN com os dois maiores Rússia e paraguai pela medida litro_total

-----------------------

Gráfico "Mulheres no mercado de trabalho da enologia"

Informação do Campo:

Distribuição de representatividade de mulheres na cultura de cultivo de Vinhos no brasil.

Pesquisa realizada pela UFRRJ com 200 mulheres envolvidas com o mundo dos vinhos, com base no resultado da pesquisa, geramos a medida de percentual e criamos a visualização de cascata.

Utilizamos a função divide, onde dividimos a quantidade pela soma total da tabela e mesmo quando o power bi faz a categorização dos valores ele mantém a soma de todos os valores da coluna quantidade trazendo assim o percentual correto.

	DIVIDE(SELECTEDVALUE(Inclusao[quantidade]), CALCULATE(SUM(Inclusao[quantidade]), ALL(Inclusao)))

-------------

Gráfico "Demográfia":

Informação do Campo:

População dos países que mais adquiriram vinhos no período, e que possuem pessoas em idade permitida por lei para o consumo de bebidas alcoolicas.

Fonte: Dados retirados do IBGE.

Como a tabela já me trazia a quantidade populacional e maior idade para consumo de álcool realizei a medida dax da multiplicação desses valores estimatindo a quantidade de pessoas que bebem na Rússia e quantidade de pessoas que bebem ou já tem idade suficiente o paraguai.

	-- Soma de população por percentual maior idade
	SUMX(populacao, populacao[populacao])*SUMX(populacao, populacao[percentual_maior_idade])

------------------

Gráfico "Economia":

Informação do Campo:

Está análise é refletida diretamente nos valores em dolares históricos, que estão sendo puxados para dentro do nosso código a partir da base de dados dolar.

Estrutura do gráfico:
Para esse gráfico categorizamos por data e para os valores puxamos a soma dinâmica da coluna último deparando os últimos valores a variação do dolar.